In [1]:
print('hello world!')

hello world!


In [4]:
%pwd

'c:\\Users\\adars\\Desktop\\AI-PROJECTS'

In [3]:
import os
os.chdir('../')

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data='Data/')

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print('Length: ',len(text_chunks))

Length:  5860


In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [11]:
def download_hugging_face_embeddings():
    from langchain_community.embeddings import HuggingFaceBgeEmbeddings
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\adars\AppData\Local\Temp\ipykernel_2780\1036191651.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\adars\Desktop\AI-PROJECTS\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
print(embeddings.embed_query("Hello World!"))
print('length : ',len(embeddings.embed_query("Hello World!")))

[0.004552410915493965, 0.17278990149497986, 0.03477634862065315, 0.005749511066824198, -0.026328887790441513, -0.04085709527134895, 0.022657936438918114, -0.0446588434278965, -0.01880376785993576, 0.008832592517137527, 0.04025248810648918, -0.034709054976701736, 0.015151108615100384, -0.014657366089522839, 0.07468860596418381, -0.04393266141414642, -0.0560385063290596, 0.020306624472141266, -0.05812908709049225, -0.04611075296998024, 0.08254793286323547, 0.10951972752809525, 0.014446125365793705, 0.025379378348588943, -0.08089374750852585, 0.015043933875858784, -0.003515304997563362, 0.013029933907091618, 0.09713464230298996, -0.061469607055187225, -0.027697697281837463, 0.0014512314228340983, 0.08353219926357269, 0.01702171005308628, -0.010031123645603657, 0.08817803859710693, 0.05356213450431824, -0.039075564593076706, 0.028452113270759583, -0.05871639400720596, 0.024071136489510536, -0.03864147886633873, -0.04744643718004227, 0.0030063141603022814, -0.0009589865221641958, -0.0254712

In [23]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'medicalbot'

pc.create_index(

    name = index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )

)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-xu7iu31.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [15]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [16]:
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    
    index_name=index_name,
    embedding=embeddings,
)

In [21]:
reriever = docsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [22]:
retrieved_docs = reriever.invoke('what is acne')

In [23]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Set up Gemini API key
genai.configure(api_key=GEMINI_API_KEY)

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.4, max_output_tokens=500,google_api_key=GEMINI_API_KEY)

In [24]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [25]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(reriever,question_answer_chain)

In [82]:
res = rag_chain.invoke({"input" : "what is stats?"})
print(res['answer'])

This question cannot be answered from the given context.  The provided text discusses blood counts and their components, not the meaning of "stats".  More information is needed to define "stats".


In [83]:
res = rag_chain.invoke({"input" : "what is acne?"})
print(res['answer'])

Acne, medically known as acne vulgaris, is a common skin disease.  It's characterized by pimples forming on the face, chest, and back due to clogged pores. These pores become blocked with oil, dead skin cells, and bacteria.


In [84]:
res = rag_chain.invoke({"input" : "what is the probability of a person surviving, sufforing from a blood cancer?"})


In [85]:
print(res['answer'])

The provided text discusses aplastic anemia, a type of blood cancer, with survival rates varying depending on treatment.  Patients receiving blood transfusions have a 20% chance of surviving 18 months to two years, while bone marrow transplants offer a 60-90% cure rate.  The text doesn't offer survival rates for other blood cancers.


In [26]:
res = rag_chain.invoke({"input" : "is stress a myth?"})
print(res['answer'])

Stress is not a myth, but a physiological response involving the release of stress hormones.  The body can sometimes trigger this response even when there's no actual external danger.  This can lead to negative biochemical side effects if the stress response is repeatedly activated without physical release.


In [ ]:
res = rag_chain.invoke({"input" : "is stress a myth?"})
print(res['answer'])

In [27]:
res = rag_chain.invoke({"input" : "what is gradient descent?"})
print(res['answer'])

This question cannot be answered from the given context. The provided text discusses Aston-Patterning, a system of movement education and bodywork, and aerospace medicine topics.  It does not contain information about gradient descent.
